In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras import Model
from keras.layers import Input, Conv2D, Activation, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
ls

In [ ]:
dir_path = 'Step 1. Data Wrangling/'

In [ ]:
df_wellA = pd.read_csv(dir_path+'DATA_for_DL_Well_A_data.csv')
df_wellA

In [ ]:
X_dataA=[]
y_dataA=[]
for ind in df_wellA.index:
    y_dataA.append(df_wellA['Porosity, pc'][ind])
    im = Image.open((dir_path+df_wellA['File Name'][ind][:-4]).replace('\\','/')+'_left.jpg')
    X_dataA.append(np.array(im))
    #print(ind,df_wellA['Porosity, pc'][ind],df_wellA['File Name'][ind])

In [ ]:
y_dataA=np.array(y_dataA)/36
y_dataA.shape

In [ ]:
X_dataA=np.array(X_dataA)/255
X_dataA.shape

In [ ]:
plt.imshow(X_dataA[1,:,:,:])

In [ ]:
plt.imshow(np.flip(X_dataA,axis=1)[1,:,:,:])

In [ ]:
plt.imshow(np.flip(X_dataA,axis=2)[1,:,:,:])

In [ ]:
X_dataA=np.concatenate((X_dataA,np.flip(X_dataA,axis=1),np.flip(X_dataA,axis=2)),axis=0)
y_dataA=np.concatenate((y_dataA,y_dataA,y_dataA),axis=0)
print(X_dataA.shape)
print(y_dataA.shape)

In [ ]:
def plot_thin_section(item):
    
    fig, ax = plt.subplots(figsize=(5, 5),frameon=False)
    
    ax.title.set_text('Porosity # '+str(round(y_dataA[item],2)))
    ax.imshow(X_dataA[item,:,:,:])

In [ ]:
sliders_item = widgets.IntSlider(min=0, max=X_dataA.shape[0], step=1, value=0, description='Item # ')
interact(plot_thin_section, item=sliders_item)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataA, y_dataA, test_size=0.10, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
def create_block(x, number_filters, filter_size=(2,2), pooling_size=(2,2), chanDim = -1):
    
    x = Conv2D(number_filters, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=pooling_size)(x)
    
    return x
    
def create_cnn(input_shape, filters=(16, 32, 64, 128), regress=False):
    
    # TensorFlow/channels-last ordering
    chanDim = -1
    
    # define the model input
    inputs = Input(shape=input_shape)
    
    b1 = create_block(inputs,32,(3, 3),(3,2))
    b2 = create_block(b1,64,(3, 3),(2,2))
    b3 = create_block(b2,128,(3, 3),(2,2))
    b4 = create_block(b3,256,(3, 3),(2,2))
    b5 = create_block(b4,512,(3, 3),(2,2))
    b6 = create_block(b5,512,(3, 3),(2,2))
    b7 = create_block(b6,512,(3, 3),(2,2))

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(b7)
    x = Dense(256)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(64)(x)
    x = Activation("relu")(x)
    
    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)
    # construct the CNN

    model = Model(inputs, x)
    
    # return the CNN
    return model

In [ ]:
model = create_cnn((X_train.shape[1], X_train.shape[2], 3), regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [ ]:
model.summary()

In [ ]:
# train the model
print("[INFO] training model...")
model.fit(X_train, y_train, 
    validation_data=(X_test, y_test),
    epochs=200, batch_size=8)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
preds = model.predict(X_test)
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds - y_test
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)
# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [ ]:
def create_cnn(input_shape, filters=(16, 32, 64, 128), regress=False):
    
    # TensorFlow/channels-last ordering
    chanDim = -1
    
    # define the model input
    inputs = Input(shape=input_shape)
    
    # loop over the number of filters
    for (i, f) in enumerate(filters):
        # if this is the first CONV layer then set the input
        # appropriately
        if i == 0:
            x = inputs
        # CONV => RELU => BN => POOL
        x = Conv2D(f, (6, 4), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(3, 2))(x)

    # flatten the volume, then FC => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(256)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
    
    # apply another FC layer, this one to match the number of nodes
    # coming out of the MLP
    x = Dense(64)(x)
    x = Activation("relu")(x)
    
    # check to see if the regression node should be added
    if regress:
        x = Dense(1, activation="linear")(x)
    # construct the CNN

    model = Model(inputs, x)
    
    # return the CNN
    return model

In [ ]:
1536, 1024

In [ ]:
1536, 1024

In [ ]:
1536, 1024

In [ ]:
df_wellbt2 = pd.read_csv('Step 1. Data Wrangling/DATA_for_DL-Well_bt2_data.csv')
df_wellbt2

In [ ]:
X_databt2=[]
y_databt2=[]
for ind in df_wellbt2.index:
    y_databt2.append(df_wellbt2['Porosity, pc'][ind])
    im = Image.open('Step 1. Data Wrangling\\'+df_wellbt2['File Name'][ind][:-4]+'_left.jpg')
    X_databt2.append(np.array(im))

In [ ]:
X_databt2=np.array(X_databt2)/255
X_databt2.shape

In [ ]:
y_databt2=np.array(y_databt2)/36
y_databt2.shape